In [1]:
import pandas as pd
import numpy as np

### Data management

In [54]:
absolute_path = '../data/data_post_comptetition/'

# to add split.json
original_dataset_path = {
    'ArMIS': absolute_path + 'ArMIS_dataset/ArMIS_',
    'ConvAbuse': absolute_path + 'ConvAbuse_dataset/ConvAbuse_',
    'Brexit': absolute_path + 'HS-Brexit_dataset/HS-Brexit_',
    'MD': absolute_path + 'MD-Agreement_dataset/MD-Agreement_',
}

dfs = {}

for name in original_dataset_path:
    dfs[name] = pd.concat([
        pd.read_json(original_dataset_path[name] + 'train.json', orient = 'index'),
        pd.read_json(original_dataset_path[name] + 'dev.json', orient = 'index'),
        pd.read_json(original_dataset_path[name] + 'test.json', orient = 'index'),
    ]).loc[:, ['text', 'number of annotations', 'hard_label', 'soft_label', 'split']]

### Extracting Level of Disagreement (LoD)

In [55]:
dfs['Brexit']

,text,number of annotations,hard_label,soft_label,split
1,<user> <user> I'm so glad about #Brexit.. My a...,6,0,"{'0': 1.0, '1': 0.0}",train
2,RT <user>: There was more to #Brexit than immi...,6,0,"{'0': 1.0, '1': 0.0}",train
3,"At the end of the day, the leave campaign won ...",6,0,"{'0': 1.0, '1': 0.0}",train
4,So the reducing migration thing wasn't quite w...,6,0,"{'0': 1.0, '1': 0.0}",train
5,A Brit Immigrant Asks Britain to Become India’...,6,0,"{'0': 1.0, '1': 0.0}",train
...,...,...,...,...,...
164,<user> Hmm... for #Brexit they can't really ch...,6,0,"{'0': 1.0, '1': 0.0}",test
165,"Sadiq, Isis Muslim mayor is a total lunatic an...",6,1,"{'0': 0.5, '1': 0.5}",test
166,Fusilier Lee Rigby&lt;----Not Forgotten During...,6,1,"{'0': 0.5, '1': 0.5}",test
167,Because the world Knows #Trump is the only one...,6,0,"{'0': 1.0, '1': 0.0}",test


In [86]:
def disagrement_level(n_annotators):
    if n_annotators % 2 == 0:
        return .5
    else:
        middle_num = n_annotators / 2
        return np.floor(middle_num)/n_annotators, np.ceil(middle_num)/n_annotators

disagrement_level(3)

(0.3333333333333333, 0.6666666666666666)

In [87]:
dfs['ArMIS']['soft_label']

1        {'0': 1.0, '1': 0.0}
2      {'0': 0.33, '1': 0.67}
3        {'0': 0.0, '1': 1.0}
4        {'0': 1.0, '1': 0.0}
5      {'0': 0.67, '1': 0.33}
                ...          
141      {'0': 1.0, '1': 0.0}
142    {'0': 0.67, '1': 0.33}
143      {'0': 1.0, '1': 0.0}
144    {'0': 0.67, '1': 0.33}
145      {'0': 1.0, '1': 0.0}
Name: soft_label, Length: 943, dtype: object

Per il confronto tra le label non usare disagrement_level == soft label ma prova ad usare la sottrazione. Alcuni numeri possono essere uguali ma approssimati diversamente:
- 0.33 != 0.333
```
if np.abs(0.3333 - 0.33) < 1e-2:
    print('numeri uguali')
else:
    print('non sono uguali')

In [28]:


dfs['Brexit']['soft_label'].apply(lambda x: 1 if (x['0'] == 0.) or (x['0'] == 1.) else 0)

1       1
2       1
3       0
4       1
5       0
       ..
3053    0
3054    0
3055    0
3056    0
3057    1
Name: soft_label, Length: 10753, dtype: int64